In [70]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModel
from torch.utils.data import Dataset, DataLoader
from sentence_transformers import SentenceTransformer, util
import random
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import Dataset, DataLoader, random_split
from transformers import BertTokenizer, BertForSequenceClassification, AdamW
from transformers import BertForSequenceClassification


In [71]:
posts = pd.read_csv('/Users/muqaddasharoon/Downloads/Thesis/fact checking claims/dataset/posts.csv')
fact_checks = pd.read_csv('/Users/muqaddasharoon/Downloads/Thesis/fact checking claims/dataset/fact_checks.csv')
pairs = pd.read_csv('/Users/muqaddasharoon/Downloads/Thesis/fact checking claims/dataset/pairs.csv')

In [72]:
fact_checks = fact_checks.sample(frac=0.1, random_state=42)
posts = posts.sample(frac=0.1, random_state=42)
pairs = pairs.sample(frac=0.1, random_state=42)


In [73]:
posts.head()

,post_id,instances,ocr,verdicts,text
2166,2398,"[(1643041167.0, 'ig')]","[(""or years the majority of people wer- onfuse...",['False'],NaN
10333,11530,"[(1643829935.0, 'fb')]",[],['Altered photo'],"('Hoy, a muy tempranas horas, el Papa Francisc..."
10835,12081,"[(1627043545.0, 'fb')]",[('Calendar week 150 8601 Number of specimens ...,['Partly false information.'],"('Influenza, finally eradicated 👏', 'Influenza..."
4688,5218,"[(1630043463.0, 'fb')]",[('Parámetros: HOLISTIC HEALING INSTITUT MEDIZ...,[],('ANALISIS DE UNA MUESTRA DE AGUA DEL GRIFO DE...
17560,19804,"[(1580851455.0, 'fb')]",[('PERÚ Ministerio de Salud INSTITUTO NACIONAL...,['False information'],('Tomemos en cuenta el Desconocimiento de much...


In [74]:

posts = posts.dropna(subset=['text'])

In [75]:
fact_checks.head()


,fact_check_id,claim,instances,title
131579,155830,"('« On aide très peu les paysans bio »', '""We ...","[(1645528774.0, 'https://www.mediacites.fr/ver...","('« On aide très peu les paysans bio »', '""We ..."
36821,42162,('Dado que ningún fabricante de las vacunas ap...,"[(1639007940.0, 'https://www.newtral.es/autori...",('La autorización de comercialización de las v...
141844,173033,"('فيديو للمسجد النبوي وهو مغطى بالثلوج.', ""A v...","[(1579534680.0, 'https://misbar.com/factcheck/...",NaN
45892,52785,('En lugar de dejar de usar las aplicaciones d...,"[(1658534280.0, 'https://www.univision.com/not...",('Apps de menstruación de Europa: lo que sabem...
3430,3543,"('""Milionário compra clube chinês e exige que ...","[(1622681063.0, 'https://observador.pt/factche...",('Fact Check. Milionário chinês obrigou clube ...


In [76]:
fact_checks['title'].fillna(fact_checks['claim'].apply(lambda x: x[:30] if isinstance(x, str) else "No Title"), inplace = True) 


In [77]:
fact_checks.head()

,fact_check_id,claim,instances,title
131579,155830,"('« On aide très peu les paysans bio »', '""We ...","[(1645528774.0, 'https://www.mediacites.fr/ver...","('« On aide très peu les paysans bio »', '""We ..."
36821,42162,('Dado que ningún fabricante de las vacunas ap...,"[(1639007940.0, 'https://www.newtral.es/autori...",('La autorización de comercialización de las v...
141844,173033,"('فيديو للمسجد النبوي وهو مغطى بالثلوج.', ""A v...","[(1579534680.0, 'https://misbar.com/factcheck/...",('فيديو للمسجد النبوي وهو مغطى
45892,52785,('En lugar de dejar de usar las aplicaciones d...,"[(1658534280.0, 'https://www.univision.com/not...",('Apps de menstruación de Europa: lo que sabem...
3430,3543,"('""Milionário compra clube chinês e exige que ...","[(1622681063.0, 'https://observador.pt/factche...",('Fact Check. Milionário chinês obrigou clube ...


In [78]:

post_ids_in_posts = set(posts['post_id'].unique())
post_ids_in_pairs = set(pairs['post_id'].unique())


missing_post_ids = post_ids_in_pairs - post_ids_in_posts #we are finding pairs that are  in posts.csv but not pairs
pairs2 = pairs[~pairs['post_id'].isin(missing_post_ids)]

fact_check_ids_in_fact_checks = set(fact_checks['fact_check_id'].unique())
fact_check_ids_in_pairs = set(pairs['fact_check_id'].unique())

missing_fact_check_ids = fact_check_ids_in_pairs - fact_check_ids_in_fact_checks

filtered_pairs = pairs[
    ~pairs['post_id'].isin(missing_post_ids) & 
    ~pairs['fact_check_id'].isin(missing_fact_check_ids)
]

merged_data = filtered_pairs.merge(posts, on='post_id', how='left')
training_data = merged_data.merge(fact_checks, on='fact_check_id', how='left')




In [79]:
import pandas as pd
import random

# Create positive pairs list
pos_pairs = []
for _, row in training_data.iterrows():
    pair_dict = row.to_dict()
    pair_dict['label'] = 1  # Positive label
    pos_pairs.append(pair_dict)

print(f"Total Positive Pairs: {len(pos_pairs)}")


fact_check_dict = fact_checks.set_index('fact_check_id').to_dict('index') 


neg_pairs = []
for _, row in training_data.iterrows():
    post_text = row['text']
    correct_fact_check_id = row['fact_check_id']

    while True:
        random_fact_id = random.choice(list(fact_check_dict.keys()))
        if random_fact_id != correct_fact_check_id:
            break

    random_fact_check_data = fact_check_dict[random_fact_id]

  
    if pd.notna(post_text) and pd.notna(random_fact_check_data['claim']):
        neg_pairs.append({
            'post_id': row['post_id'],  
            'fact_check_id': random_fact_id,  
            'instances_x': row['instances_x'],  
            'ocr': row['ocr'], 
            'verdicts': row['verdicts'], 
            'text': post_text,  
            'claim': random_fact_check_data['claim'], 
            'instances_y': random_fact_check_data['instances'],
            'title': random_fact_check_data['title'],  
            'label': 0  
        })

print(f"Total Negative Samples: {len(neg_pairs)}")
print(neg_pairs[0]) 
all_pairs = pos_pairs + neg_pairs


Total Positive Pairs: 27
Total Negative Samples: 27
{'post_id': 23023, 'fact_check_id': 108741, 'instances_x': "[(1570019069.0, 'fb')]", 'ocr': '[("Gerard Depardieu #pourquoi la Tunisie? 40 ? Pourquoi la Tunisie Je ne suis pas un politicien et je déteste les politiciens, car la plupart d\'entre eux sont des hypocrites corrompus et flatteurs. Encore une fois, ces politiciens interviennent en Tunisie et aident les corrompus comme eux: pourquoi ne laissez-vous pas le peuple tunisien décider de son sort? Pour vivre comme les autres peuples du monde, pourquoi insistez-vous pour faire de la Tunisie la gueule de vos déchets et un foyer de personnes corrompues? Quittez les peuples et quittez leur pays d\'origine afin que nous ne payions pas vos erreurs dans nos pays d\'origine. Je vais garder cette fougue odieuse vous jamais.", "Gérard Depardieu #why Tunisia? 40 ? Why Tunisia I am not a politician and I hate politicians because most of them they are corrupt and flattering hypocrites. Again, th

In [80]:
df = pd.DataFrame(all_pairs)
df = df.drop(columns = ['post_id', 'fact_check_id', 'instances_x','instances_y' ])


In [81]:
df.head()

,ocr,verdicts,text,claim,title,label
0,"[(""Gerard Depardieu #pourquoi la Tunisie? 40 ?...",['False information'],"('الممثل الفرنسي "" Gerard Depardieu"" نطق واعتر...",('الممثل الفرنسي Gerard Depardieu نطق واعترف أ...,"('هذه الصورة للممثّل جيرار دوبارديو مركّبة', '...",1
1,[('MARVIN BERGAUER REPORTER OE24TV WIEN: DEMO ...,['Missing context'],"('😂 Milagros del cambio plandémico... 👇', '😂 M...",('Vídeo de un informativo en el que un cadáver...,"('No, este vídeo de un informativo austriaco e...",1
2,[],[],('GRAN FRASE 🤺 “Querido Sancho: Compruebo con ...,('Sabio Don Miguel de Cervantes. “Querido Sanc...,"(""Querido Sancho compruebo con pesar...: la ci...",1
3,[],['Missing context.'],('Tarian klasik ini baru diciptakan di China d...,('Video penari robot buatan Tiongkok tampil di...,"(""Penari di video ini adalah manusia, bukan 'r...",1
4,[],['False information'],('जाफ़राबाद नई दिल्ली के दृश्य.. लगता है प्रधा...,"(""Video shows COVID protocol flouted in Delhi'...",('Video From Lahore Passed Off as COVID-19 Nor...,1


In [84]:
import torch
from transformers import BertTokenizer, EncoderDecoderModel, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
import pandas as pd


train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)


tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')


def preprocess_data(ocr, verdicts, text):
    combined_input = ocr + " [SEP] " + verdicts + " [SEP] " + text
    return combined_input

# Concatenate relevant information for both train and test
train_inputs = [preprocess_data(a, b, c) for a, b, c in zip(train_df['ocr'], train_df['verdicts'], train_df['text'])]
test_inputs = [preprocess_data(a, b, c) for a, b, c in zip(test_df['ocr'], test_df['verdicts'], test_df['text'])]

# Tokenize input text (input sentences) and output text (claims)
train_encodings = tokenizer(train_inputs, padding="max_length", truncation=True, max_length=128, return_tensors='pt')
test_encodings = tokenizer(test_inputs, padding="max_length", truncation=True, max_length=128, return_tensors='pt')

# Tokenize the claims (outputs)
train_claims_encodings = tokenizer(train_df['claim'].tolist(), padding="max_length", truncation=True, max_length=128, return_tensors='pt')
test_claims_encodings = tokenizer(test_df['claim'].tolist(), padding="max_length", truncation=True, max_length=128, return_tensors='pt')


class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: val[idx] for key, val in self.encodings.items()}
        item['labels'] = self.labels['input_ids'][idx]  # Use the input_ids of the target (claim) as labels
        return item

    def __len__(self):
        return len(self.labels['input_ids'])

train_dataset = CustomDataset(train_encodings, train_claims_encodings)
test_dataset = CustomDataset(test_encodings, test_claims_encodings)





In [86]:
from transformers import BertTokenizer, EncoderDecoderModel
import torch
from torch.utils.data import DataLoader


model = EncoderDecoderModel.from_encoder_decoder_pretrained('bert-base-multilingual-cased', 'bert-base-multilingual-cased')

model.config.decoder_start_token_id = tokenizer.cls_token_id


model.config.pad_token_id = tokenizer.pad_token_id


device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)


optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=16)

# Training loop
num_epochs = 3

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for batch in train_loader:
        inputs = {key: val.to(device) for key, val in batch.items() if key != 'labels'}
        labels = batch['labels'].to(device)

        outputs = model(**inputs, labels=labels)
        loss = outputs.loss

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        total_loss += loss.item()

    avg_train_loss = total_loss / len(train_loader)
    print(f"Epoch {epoch + 1}/{num_epochs}, Loss: {avg_train_loss}")


    model.eval()
    with torch.no_grad():
        # Implement your evaluation loop here if needed
        pass

Some weights of BertLMHeadModel were not initialized from the model checkpoint at bert-base-multilingual-cased and are newly initialized: ['bert.encoder.layer.0.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.0.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.0.crossattention.output.dense.bias', 'bert.encoder.layer.0.crossattention.output.dense.weight', 'bert.encoder.layer.0.crossattention.self.key.bias', 'bert.encoder.layer.0.crossattention.self.key.weight', 'bert.encoder.layer.0.crossattention.self.query.bias', 'bert.encoder.layer.0.crossattention.self.query.weight', 'bert.encoder.layer.0.crossattention.self.value.bias', 'bert.encoder.layer.0.crossattention.self.value.weight', 'bert.encoder.layer.1.crossattention.output.LayerNorm.bias', 'bert.encoder.layer.1.crossattention.output.LayerNorm.weight', 'bert.encoder.layer.1.crossattention.output.dense.bias', 'bert.encoder.layer.1.crossattention.output.dense.weight', 'bert.encoder.layer.1.crossattention.self.key.bia

Epoch 1/3, Loss: 11.256892999013266
Epoch 2/3, Loss: 6.579164028167725
Epoch 3/3, Loss: 4.618646105130513
